In [53]:
rawdata = spark.read.options(sep="\t").csv("s3://tf-idf-data/subset-small.tsv")
rawdata.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------------------+-------------------+--------------------+
|_c0|                 _c1|                _c2|                 _c3|
+---+--------------------+-------------------+--------------------+
| 12|           Anarchism|2008-12-30 06:23:05|Anarchism (someti...|
| 25|              Autism|2008-12-24 20:41:05|Autism is a brain...|
| 39|              Albedo|2008-12-29 18:19:09|The albedo of an ...|
|290|                   A|2008-12-27 04:33:16|The letter A is t...|
|303|             Alabama|2008-12-29 08:15:47|Alabama (formally...|
|305|            Achilles|2008-12-30 06:18:01|thumb\n\nIn Greek...|
|307|     Abraham Lincoln|2008-12-28 20:18:23|Abraham Lincoln (...|
|308|           Aristotle|2008-12-29 23:54:48|Aristotle (Greek:...|
|309|An American in Paris|2008-09-27 19:29:28|An American in Pa...|
|324|       Academy Award|2008-12-28 17:50:43|The Academy Award...|
|330|             Actrius|2008-05-23 15:24:32|Actrius (Actresse...|
|332|     Animalia (book)|2008-12-18 11:12:34|th

In [55]:
rawdata.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1000

In [3]:
articles = rawdata.toDF("ID", "Name", "Time", "Document")
articles.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------------------+-------------------+--------------------+
| ID|                Name|               Time|            Document|
+---+--------------------+-------------------+--------------------+
| 12|           Anarchism|2008-12-30 06:23:05|Anarchism (someti...|
| 25|              Autism|2008-12-24 20:41:05|Autism is a brain...|
| 39|              Albedo|2008-12-29 18:19:09|The albedo of an ...|
|290|                   A|2008-12-27 04:33:16|The letter A is t...|
|303|             Alabama|2008-12-29 08:15:47|Alabama (formally...|
|305|            Achilles|2008-12-30 06:18:01|thumb\n\nIn Greek...|
|307|     Abraham Lincoln|2008-12-28 20:18:23|Abraham Lincoln (...|
|308|           Aristotle|2008-12-29 23:54:48|Aristotle (Greek:...|
|309|An American in Paris|2008-09-27 19:29:28|An American in Pa...|
|324|       Academy Award|2008-12-28 17:50:43|The Academy Award...|
|330|             Actrius|2008-05-23 15:24:32|Actrius (Actresse...|
|332|     Animalia (book)|2008-12-18 11:12:34|th

In [4]:
articles.filter(articles.Document.isNull()).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1

In [6]:
cleanedData = articles.filter(articles.Document.isNotNull())
cleanedData.filter(articles.Document.isNull()).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [8]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

tokenizer= Tokenizer(inputCol="Document", outputCol="Tokens")
TokenData = tokenizer.transform(cleanedData)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
TokenData.select("Document").show(5,truncate=True), TokenData.select("Tokens").show(5,truncate=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|            Document|
+--------------------+
|Anarchism (someti...|
|Autism is a brain...|
|The albedo of an ...|
|The letter A is t...|
|Alabama (formally...|
+--------------------+
only showing top 5 rows

+--------------------+
|              Tokens|
+--------------------+
|[anarchism, (some...|
|[autism, is, a, b...|
|[the, albedo, of,...|
|[the, letter, a, ...|
|[alabama, (formal...|
+--------------------+
only showing top 5 rows

(None, None)

In [26]:
hashingTF = HashingTF(inputCol="Tokens", outputCol="FeatureVectors")
featurizedData = hashingTF.transform(TokenData)
featurizedData.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------------------+-------------------+--------------------+--------------------+--------------------+
| ID|                Name|               Time|            Document|              Tokens|      FeatureVectors|
+---+--------------------+-------------------+--------------------+--------------------+--------------------+
| 12|           Anarchism|2008-12-30 06:23:05|Anarchism (someti...|[anarchism, (some...|(262144,[116,120,...|
| 25|              Autism|2008-12-24 20:41:05|Autism is a brain...|[autism, is, a, b...|(262144,[521,1546...|
| 39|              Albedo|2008-12-29 18:19:09|The albedo of an ...|[the, albedo, of,...|(262144,[1625,179...|
|290|                   A|2008-12-27 04:33:16|The letter A is t...|[the, letter, a, ...|(262144,[5303,603...|
|303|             Alabama|2008-12-29 08:15:47|Alabama (formally...|[alabama, (formal...|(262144,[93,115,3...|
|305|            Achilles|2008-12-30 06:18:01|thumb\n\nIn Greek...|[thumb\n\nin, gre...|(262144,[305,991,...|
|307|     

In [28]:
idf = IDF(inputCol="FeatureVectors", outputCol="Features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
rescaledData.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
| ID|                Name|               Time|            Document|              Tokens|      FeatureVectors|            Features|
+---+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
| 12|           Anarchism|2008-12-30 06:23:05|Anarchism (someti...|[anarchism, (some...|(262144,[116,120,...|(262144,[116,120,...|
| 25|              Autism|2008-12-24 20:41:05|Autism is a brain...|[autism, is, a, b...|(262144,[521,1546...|(262144,[521,1546...|
| 39|              Albedo|2008-12-29 18:19:09|The albedo of an ...|[the, albedo, of,...|(262144,[1625,179...|(262144,[1625,179...|
|290|                   A|2008-12-27 04:33:16|The letter A is t...|[the, letter, a, ...|(262144,[5303,603...|(262144,[5303,603...|
|303|             Alabama|2008-12-29 08:15:47|Alabama (formally...|[alabama, (forma

In [41]:
from pyspark.sql.types import *

schema = StructType([StructField("Tokens", ArrayType(StringType()))])

df = spark.createDataFrame(([[["abraham"]]]), schema).toDF("Tokens")
df.show()



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+
|   Tokens|
+---------+
|[abraham]|
+---------+

In [46]:
abraham = hashingTF.transform(df)
abraham.show()



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+
|   Tokens|      FeatureVectors|
+---------+--------------------+
|[abraham]|(262144,[206934],...|
+---------+--------------------+

In [47]:
featureVec = abraham.select('FeatureVectors').collect()
print(featureVec)

abrahamID = int(featureVec[0].FeatureVectors.indices[0])
print(abrahamID)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(FeatureVectors=SparseVector(262144, {206934: 1.0}))]
206934

In [48]:
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf

termExtractor = udf(lambda x: float(x[abrahamID]), FloatType())
abrahamDF = rescaledData.withColumn('abraham_score', termExtractor(rescaledData.Features))

abrahamDF.show()
                          

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+-------------+
| ID|                Name|               Time|            Document|              Tokens|      FeatureVectors|            Features|abraham_score|
+---+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+-------------+
| 12|           Anarchism|2008-12-30 06:23:05|Anarchism (someti...|[anarchism, (some...|(262144,[116,120,...|(262144,[116,120,...|          0.0|
| 25|              Autism|2008-12-24 20:41:05|Autism is a brain...|[autism, is, a, b...|(262144,[521,1546...|(262144,[521,1546...|          0.0|
| 39|              Albedo|2008-12-29 18:19:09|The albedo of an ...|[the, albedo, of,...|(262144,[1625,179...|(262144,[1625,179...|          0.0|
|290|                   A|2008-12-27 04:33:16|The letter A is t...|[the, letter, a, ...|(262144,[5303,603...|(262144,[5303,603...|

In [50]:
sortedResults = abrahamDF.filter("abraham_score > 0").orderBy('abraham_score', ascending=False).select('ID', 'Name', 'Document', 'abraham_score')
sortedResults.show(truncate=100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------------------------+----------------------------------------------------------------------------------------------------+-------------+
|  ID|                         Name|                                                                                            Document|abraham_score|
+----+-----------------------------+----------------------------------------------------------------------------------------------------+-------------+
|1436|                      Abraham|thumb Abraham (  Avrohom or Avruhom ; ,  ; Ge'ez: , ) is a man featured in the Book of Genesis an...|     324.6979|
| 307|              Abraham Lincoln|Abraham Lincoln (February 12, 1809 – April 15, 1865) was the sixteenth President of the United St...|    46.944275|
|1624|               Andrew Johnson|Andrew Johnson (December 29, 1808 – July 31, 1875) was the seventeenth President of the United St...|    15.648092|
| 863|           American Civil War|The American Civil War (1861–1865), also known as th